# LSTM情感分类
英文长影评数据集, 共1400条数据

一共8万+个词, 每个文本最长2493个词, 模型的效果很差只有0.5, 或许是因为这个, 数据量太小

In [1]:
import numpy as np
import os
import torch
from nltk.stem.lancaster import LancasterStemmer
import keras.preprocessing.text as T
import re
from gensim import corpora
from keras.layers import Dense, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.model_selection import train_test_split

D:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\study\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 载入数据

In [9]:
def loadDataSet(route, category):
    files = os.listdir(route)   # 正类影评
    
    X = []; y = []

    for file in files:
        with open(route + file, 'r', encoding='UTF-8-sig') as f:
            content = f.read()
            X.append(content)
            y.append(category)
    return X, y

In [15]:
pos_X, pos_y = loadDataSet(r'D:\CS\dataset\NLP\IMDB Movie Reviews Dataset\aclImdb\train\pos\\', 1)
neg_X, neg_y = loadDataSet(r'D:\CS\dataset\NLP\IMDB Movie Reviews Dataset\aclImdb\train\neg\\', 0)   

pos_X.extend(neg_X) ; pos_y.extend(neg_y)
X = pos_X.copy() ; y = pos_y.copy()

In [16]:
def token(X):   # 将X中文本分词
    """
    Args:
        X: [text0, text1, text2]
    Returns:
        text_list: [['you', 'and', ...], ['what', 'honey', ...]]
    """
    
    text_list = []
    for text in X:
        text_list.append(T.text_to_word_sequence(text))
    return text_list

In [17]:
X = token(X)
X[0]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 '35',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me',
 'to',
 'believe',
 'that',
 'bromwell',
 "high's",
 'satire',
 'is',
 'much',
 'closer',
 'to',
 'reality',
 'than',
 'is',
 'teachers',
 'the',
 'scramble',
 'to',
 'survive',
 'financially',
 'the',
 'insightful',
 'students',
 'who',
 'can',
 'see',
 'right',
 'through',
 'their',
 'pathetic',
 "teachers'",
 'pomp',
 'the',
 'pettiness',
 'of',
 'the',
 'whole',
 'situation',
 'all',
 'remind',
 'me',
 'of',
 'the',
 'schools',
 'i',
 'knew',
 'and',
 'their',
 'students',
 'when',
 'i',
 'saw',
 'the',
 'episode',
 'in',
 'which',
 'a',
 'student',
 'repeatedly',
 'tried',
 'to',
 'burn',
 'down',
 'the',
 'school',
 'i',
 'immediately',
 'recalled',
 'at',
 'high',
 'a',
 'classic',
 'line',
 'insp

In [18]:
# 建立字典
dictionary = corpora.Dictionary(X)

In [19]:
# 获取每个文本的最大词数
length = 0
for doc in X:
    if len(doc) > length:
        length = len(doc)

num_word = max(dictionary.token2id.values()) + 1
print('最长文本的词数为: %d' %length)
print('总词数为: %d' %num_word)

最长文本的词数为: 2493
总词数为: 88582


In [20]:
def docs2idx(docs, dictionary):
    temp_list = []
    for doc in docs:
        temp_list.append(dictionary.doc2idx(doc))
        
    return temp_list

sequence_X = docs2idx(X, dictionary)

In [38]:
# 对词ID序列文本padding
from keras.preprocessing import sequence

padded_sequence_X = sequence.pad_sequences(sequence_X, length, padding='post', value=num_word+1)

In [39]:
# 划分训练, 测试集
train_X, test_X, train_y, test_y = train_test_split(padded_sequence_X, y, test_size=0.2)

## Model

In [40]:
embedding_size = 32
hidden_layer_size = 32
batch_size = 1000

model = Sequential()
model.add(Embedding(num_word+2, output_dim=embedding_size))
model.add(LSTM(hidden_layer_size, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
model.fit(train_X, train_y, epochs=10, batch_size=batch_size)

Epoch 1/10
 1000/20000 [>.............................] - ETA: 33:59 - loss: 0.6932 - acc: 0.5010